In [1]:
import yaml
import os
import pprint
from copy import deepcopy
import pandas as pd
from pathlib import Path
from influence_moo.config import load_config, consolidate_parameters, create_directory_dict, expand_keys, expand_directory_dict, write_directory_dict

In [3]:
sweep_config_dir = '~/influence-multi-objective/example_sweep/sweep.yaml'
top_dir = "~/influence-multi-objective/results/auto-generated/"

# Load in sweep config
sweep_config = load_config(sweep_config_dir)
expand_keys(sweep_config);

In [4]:
consolidated_dict = consolidate_parameters(sweep_config['parameter_dicts'])
directory_dict = create_directory_dict(consolidated_dict, path_len=len(sweep_config['parameter_dicts'])-1)
expand_directory_dict(directory_dict, sweep_config['base_dict'])
write_directory_dict(directory_dict, top_dir)

In [5]:
for dir in directory_dict:
    write_dir = Path(os.path.expanduser(top_dir)) / dir
    if not os.path.exists(write_dir):
        os.makedirs(write_dir, exist_ok = True)
    with open(write_dir/'config.yaml', 'w') as file:
        yaml.dump(directory_dict[dir], file, default_flow_style=False)  # `default_flow_style=False` makes the output more readable